In [22]:
import preprocess.prepare as prepare
import preprocess.word_bag as word_bag
import model.naive_bayes
import model.nn
import numpy as np
from utils.cache import cached
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data = cached(lambda:prepare.CratesData(), "crates_data.pkl")

Loading cached object from .cs229_cache\crates_data.pkl


In [3]:
crates = [crate for crate in data.id2crates.values() if crate.category_indices]
len(crates)

32225

In [4]:
len(crates)

32225

In [5]:
dic = cached(lambda:word_bag.create_dictionary(crates), "dictionary.pkl")

Loading cached object from .cs229_cache\dictionary.pkl


In [6]:
X, y = cached(lambda: word_bag.transform_crate(crates[:10000], dic, len(data.categories)), "wb10000.pkl")

Loading cached object from .cs229_cache\wb10000.pkl


In [23]:
model = model.nn.DNNModel(num_categories=len(data.categories), learning_rate=0.0001, reg=0.000)

In [24]:
model.fit(X, y, epochs=10)

Epoch: 0, Loss: 0.0070, val_loss: 0.0062
Epoch: 1, Loss: 0.0040, val_loss: 0.0032
Epoch: 2, Loss: 0.0025, val_loss: 0.0026
Epoch: 3, Loss: 0.0020, val_loss: 0.0024
Epoch: 4, Loss: 0.0018, val_loss: 0.0024
Epoch: 5, Loss: 0.0018, val_loss: 0.0024
Epoch: 6, Loss: 0.0016, val_loss: 0.0024
Epoch: 7, Loss: 0.0016, val_loss: 0.0021
Epoch: 8, Loss: 0.0014, val_loss: 0.0019
Epoch: 9, Loss: 0.0012, val_loss: 0.0017


In [10]:
y_hat = model.predict(X)

In [11]:
y_pred = (y_hat > 0).astype(np.float32)

In [21]:
y_pred[1024]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [20]:
y[1024]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])